In [11]:
!pip install wordfreq
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   -------------------------------- ------- 9.7/11.8 MB 60.1 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.8 MB 60.0 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 25.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   --------------------------------------- 632.6/632.6 kB 24.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 31.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 54.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import brown, gutenberg
nltk.download('brown')
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import requests
import json
import time
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy
import re




[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
# Create test sentences with known slang
test_sentences_with_slang = [
    "That party last night was lit, everyone had a great time.",
    "He ghosted me after our date last week.",
    "This project is going to be dope when it's finished.",
    "I'm totally vibing with this new music.",
    "The teacher was throwing shade at the students who didn't do their homework.",
    "That's sus, I don't believe you.",
    "The new iPhone is fire, I can't wait to get one.",
    "Stop flexing your new car on social media."
]

# Create control sentences without slang
test_sentences_without_slang = [
    "The weather is quite pleasant today.",
    "I need to finish my homework before tomorrow.",
    "The movie received positive reviews from critics.",
    "She plays tennis every Saturday morning.",
    "They decided to renovate their kitchen last summer."
]

# Combine into a test dataset
test_data = pd.DataFrame({
    'text': test_sentences_with_slang + test_sentences_without_slang,
    'has_slang': [1]*len(test_sentences_with_slang) + [0]*len(test_sentences_without_slang)
})

print("Test dataset:")
print(test_data)


Test dataset:
                                                 text  has_slang
0   That party last night was lit, everyone had a ...          1
1             He ghosted me after our date last week.          1
2   This project is going to be dope when it's fin...          1
3             I'm totally vibing with this new music.          1
4   The teacher was throwing shade at the students...          1
5                    That's sus, I don't believe you.          1
6    The new iPhone is fire, I can't wait to get one.          1
7          Stop flexing your new car on social media.          1
8                The weather is quite pleasant today.          0
9       I need to finish my homework before tomorrow.          0
10  The movie received positive reviews from critics.          0
11           She plays tennis every Saturday morning.          0
12  They decided to renovate their kitchen last su...          0


In [ ]:
# Approach 1: Using spaCy with WordNet for slang detection


# !python -m spacy download en_core_web_sm

print("\n--- Approach 1: spaCy with WordNet ---")

try:
    # Load spaCy model
    nlp = spacy.load("en_core_web_sm")
    
    def detect_slang_spacy_wordnet(text):
        """
        Detect potential slang words using spaCy and WordNet
        
        Strategy:
        1. Use spaCy for tokenization and POS tagging
        2. Check if the word exists in WordNet
        3. Words not in WordNet and not proper nouns are potential slang
        """
        doc = nlp(text)
        results = []
        
        for token in doc:
            # Skip punctuation, numbers, and short words
            if not token.is_alpha or len(token.text) <= 2:
                continue
                
            # Skip proper nouns and stop words
            if token.pos_ == "PROPN" or token.is_stop:
                continue
                
            word = token.text.lower()
            
            # Check if word exists in WordNet
            synsets = wordnet.synsets(word)
            
            # If word doesn't exist in WordNet, it might be slang
            if not synsets:
                results.append({
                    "word": token.text,
                    "pos": token.pos_,
                    "confidence": 0.7
                })
        
        return results
    
    # Test on our dataset
    for i, row in test_data.iterrows():
        slang_words = detect_slang_spacy_wordnet(row['text'])
        print(f"Sentence: {row['text']}")
        print(f"Has slang (ground truth): {row['has_slang']}")
        print(f"Detected potential slang: {[item['word'] for item in slang_words]}")
        print("---")
        
except Exception as e:
    print(f"Error loading spaCy: {e}")
    print("Skipping spaCy approach")


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 3.4 MB/s eta 0:00:04
     --------------------------------------  12.6/12.8 MB 43.7 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 42.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

--- Approach 1: spaCy with WordNet ---



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Sentence: That party last night was lit, everyone had a great time.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: He ghosted me after our date last week.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: This project is going to be dope when it's finished.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: I'm totally vibing with this new music.
Has slang (ground truth): 1
Detected potential slang: ['vibing']
---
Sentence: The teacher was throwing shade at the students who didn't do their homework.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: That's sus, I don't believe you.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: The new iPhone is fire, I can't wait to get one.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: Stop flexing your new car on social media.
Has slang (ground truth): 1
Detected potential slang: []
---
Sentence: The weather is quite 

In [20]:
# Approach 2: Zero-shot classification with BART

nltk.download('punkt_tab') # download for word_tokenize

print("\n--- Approach 2: Zero-shot Classification with BART ---")

try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    
    # Function to identify potential slang words
    def identify_slang_words_zero_shot(text, classifier):
        # First classify the whole sentence
        sentence_result = classifier(text, ["contains slang", "formal language"])
        sentence_has_slang = sentence_result['labels'][0] == "contains slang"
        
        if not sentence_has_slang:
            return []
        
        # If sentence likely has slang, check individual words
        words = word_tokenize(text)
        words = [word for word in words if word.isalpha() and len(word) > 2]
        
        results = []
        for word in words:
            # Create a simple context to check if the word is slang
            context = f"The word '{word}' is"
            word_result = classifier(context, ["slang", "standard English"])
            
            if word_result['labels'][0] == "slang" and word_result['scores'][0] > 0.9:
                results.append({
                    "word": word,
                    "confidence": word_result['scores'][0]
                })
        
        return results
    
    # Test on a few examples (this can be slow)
    for sentence in test_data['text'][:3]:  # Just test first 3 to save time
        print(f"Sentence: {sentence}")
        slang_words = identify_slang_words_zero_shot(sentence, classifier)
        print(f"Detected slang: {[item['word'] for item in slang_words]}")
        print("---")
        
except Exception as e:
    print(f"Error with zero-shot classification: {e}")
    print("Skipping zero-shot approach")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



--- Approach 2: Zero-shot Classification with BART ---


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Sentence: That party last night was lit, everyone had a great time.
Detected slang: []
---
Sentence: He ghosted me after our date last week.
Detected slang: ['ghosted']
---
Sentence: This project is going to be dope when it's finished.
Detected slang: ['dope']
---


In [21]:
# Approach 3: Using HuggingFace's text-classification models
print("\n--- Approach 3: Text Classification with HuggingFace ---")

try:
    # Load a sentiment analysis model (as a proxy for formality detection)
    sentiment_analyzer = pipeline("sentiment-analysis")
    
    def detect_slang_with_sentiment(text):
        """
        Use sentiment analysis as a proxy for detecting informal language
        """
        # Analyze whole sentence
        result = sentiment_analyzer(text)
        
        # Tokenize and check individual words
        words = word_tokenize(text)
        words = [word for word in words if word.isalpha() and len(word) > 2]
        
        # Check each word in context
        results = []
        for word in words:
            # Skip common words
            if word.lower() in ['the', 'and', 'that', 'this', 'with', 'from', 'have', 'they']:
                continue
                
            # Create contexts to test the word
            formal_context = f"The {word} is appropriate for formal writing."
            informal_context = f"The {word} is slang."
            
            formal_result = sentiment_analyzer(formal_context)[0]
            informal_result = sentiment_analyzer(informal_context)[0]
            
            # If the word is more likely to be described as slang
            if (informal_result['label'] == 'POSITIVE' and 
                informal_result['score'] > 0.7 and
                formal_result['label'] == 'NEGATIVE'):
                
                results.append({
                    "word": word,
                    "confidence": informal_result['score']
                })
        
        return results
    
    # Test on a few examples
    for sentence in test_data['text'][:3]:  # Just test first 3 to save time
        print(f"Sentence: {sentence}")
        slang_words = detect_slang_with_sentiment(sentence)
        print(f"Detected slang: {[item['word'] for item in slang_words]}")
        print("---")
        
except Exception as e:
    print(f"Error with sentiment analysis: {e}")
    print("Skipping sentiment analysis approach")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



--- Approach 3: Text Classification with HuggingFace ---


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Sentence: That party last night was lit, everyone had a great time.
Detected slang: []
---
Sentence: He ghosted me after our date last week.
Detected slang: []
---
Sentence: This project is going to be dope when it's finished.
Detected slang: []
---


: 

In [ ]:
# Approach 4: Using GPT API for slang detection
print("\n--- Approach 4: Using GPT API ---")
print("Note: This requires an OpenAI API key. Showing example code only.")

def detect_slang_with_gpt(text, api_key=None):
    """
    Use GPT API to detect slang words in text
    
    Note: This is example code and requires an API key to run
    """
    if not api_key:
        return "API key required"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    prompt = f"""
    Analyze the following text and identify any slang words or phrases:
    
    "{text}"
    
    For each slang word or phrase, provide:
    1. The word or phrase
    2. A brief explanation of what it means
    3. A confidence score (0-1) of how certain you are it's slang
    
    Format your response as a JSON object with a list of identified slang terms.
    """
    
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "response_format": {"type": "json_object"}
    }
    
    # This would make the actual API call
    # response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)
    # return response.json()
    
    # Instead, return example output
    example_output = {
        "slang_terms": [
            {
                "term": "lit",
                "meaning": "Exciting, excellent, or amazing",
                "confidence": 0.95
            },
            {
                "term": "ghosted",
                "meaning": "When someone cuts off all communication without explanation",
                "confidence": 0.98
            }
        ]
    }
    
    return example_output

# Show example output for GPT API
print("Example GPT API output for 'That party last night was lit, everyone ghosted early though.':")
example_result = detect_slang_with_gpt("That party last night was lit, everyone ghosted early though.")
print(json.dumps(example_result, indent=2))


In [ ]:
# Approach 5: Using a dictionary-based approach with common slang terms
print("\n--- Approach 5: Dictionary of Common Slang Terms ---")

# Create a small dictionary of common slang terms
common_slang = {
    "lit": "Exciting or excellent",
    "dope": "Excellent, very good",
    "fire": "Excellent or exciting",
    "ghosted": "When someone cuts off all communication",
    "vibing": "Enjoying or connecting with something",
    "shade": "Subtle disrespect or criticism",
    "sus": "Suspicious or questionable",
    "flex": "To show off",
    "salty": "Bitter or angry",
    "cap": "A lie or to lie",
    "bet": "Agreement or affirmation",
    "fam": "Friends or family",
    "goat": "Greatest of all time",
    "lowkey": "Secretly or subtly",
    "highkey": "Obviously or clearly",
    "slay": "To do something exceptionally well",
    "tea": "Gossip or information",
    "yeet": "To throw something forcefully",
    "woke": "Aware of social issues",
    "basic": "Mainstream or unoriginal"
}

def detect_slang_dictionary(text, slang_dict):
    """
    Detect slang words using a dictionary of common slang terms
    """
    # Tokenize the text
    words = word_tokenize(text.lower())
    
    # Find matches in our slang dictionary
    results = []
    for word in words:
        if word in slang_dict:
            results.append({
                "word": word,
                "meaning": slang_dict[word],
                "confidence": 1.0  # High confidence for dictionary matches
            })
    
    return results

# Test on our dataset
for i, row in test_data.iterrows():
    slang_words = detect_slang_dictionary(row['text'], common_slang)
    print(f"Sentence: {row['text']}")
    print(f"Has slang (ground truth): {row['has_slang']}")
    print(f"Detected slang: {[item['word'] for item in slang_words]}")
    print("---")

# Evaluate the dictionary approach
test_data['detected_slang'] = test_data['text'].apply(lambda x: detect_slang_dictionary(x, common_slang))
test_data['predicted_has_slang'] = test_data['detected_slang'].apply(lambda x: len(x) > 0)
accuracy = (test_data['has_slang'] == test_data['predicted_has_slang']).mean()
print(f"Dictionary approach accuracy: {accuracy:.2f}")



In [ ]:
# Summary of approaches
print("\n--- Summary of Slang Detection Approaches ---")
print("""
1. spaCy with WordNet:
   - Pros: No external API needed, linguistically informed
   - Cons: May flag legitimate rare words as slang, misses context

2. Zero-shot Classification with BART:
   - Pros: Can detect slang without explicit training
   - Cons: Slow, less accurate for individual words

3. Text Classification with HuggingFace:
   - Pros: Uses pre-trained models
   - Cons: Models not specifically trained for slang detection

4. GPT API:
   - Pros: High accuracy, provides explanations
   - Cons: Requires API key, costs money, slower

5. Dictionary-based Approach:
   - Pros: Fast, simple, high precision
   - Cons: Limited to known slang terms, requires manual updates

Recommendation:
For a production system, a hybrid approach combining dictionary lookup with
either GPT API or a fine-tuned BERT model would likely provide the best results.
""")